## CT Pedestal Computation (Optional Feature)

This notebook appends four pedestal landmarks using CT fiducial coordinates that are hardcoded into the QC pipeline so they remain available during reprojection analysis.

### Marker Mapping

- **Nose tip**: Fiducial `F` aligns with the tracked nose landmark (LPS mm coordinates: `[15.0423, 86.4624, 114.5245]`).
- **Pedestal corners**: `F_1` … `F_4` are the pedestal reference points from the same CT scan.

### How It Works

1. **Hardcoded Offsets**: Each pedestal offset `pedestal_position - nose_position` is precomputed from the embedded CT coordinates.
2. **Pedestal Trajectories**: On every frame the offset is added to the tracked nose 3D position to generate `Pedestal_F_1` … `Pedestal_F_4`.
3. **Integration**: The extra joints are appended to the 3D array and flow through all reprojection calculations.

### Configuration

- Set `nose_landmark_name` in the code cell to the tracked nose landmark name (e.g., `"F"`).
- Toggle `include_ct_pedestals` to `False` if you want to suppress the derived pedestal joints.

### Reprojection Error Note

The pedestal landmarks are derived from the tracked nose and have no independent 2D tracks. Their reprojection rows will report `n_obs = 0` and `NaN` errors, but the projected locations remain helpful for visual checks inside each camera view.


In [ ]:
"""
PHASE 2 (Reprojection Error QC)
--------------------------------
For every frame t, every joint j, every camera c:
    1. Take 3D point (X,Y,Z) in world / ref camera coordinates.
    2. Project into camera c using that camera's 3x4 P matrix.
    3. Compare predicted 2D (px) vs actual tracked 2D (px).
    4. Euclidean distance in pixels = reprojection error.

We report median and p95 reprojection error per (camera, joint),
plus an overall summary per joint across cameras.
"""

import os, re, json
import numpy as np
import pandas as pd
import h5py

# ===================== USER PATHS (EDIT THESE) =====================

# 3D file (either .h5 with "tracks" or .npz with (T,J,3))
path_3d    = r"../data/phase2_sample\points3d.h5"

# Matching 2D tracking files (the *.inference.analysis.h5 per camera)
# IMPORTANT: keys here MUST match the camera names in calibration["P"]
cam2d_files = {
    "cam-bottomleft.mp4":   r"../data/phase2_sample\cam-bottomleft.inference.analysis.h5",
    "cam-bottomright.mp4":  r"../data/phase2_sample\cam-bottomright.inference.analysis.h5",
    "cam-topleft.mp4":      r"../data/phase2_sample\cam-topleft.inference.analysis.h5",
    "cam-topright.mp4":     r"../data/phase2_sample\cam-topright.inference.analysis.h5",
}

# calibration.json that has P[camera] = 3x4 matrix
path_calib = r"../data/phase2_sample\calibration.json"

# Output folder
out_root   = r"../results"
os.makedirs(out_root, exist_ok=True)

# Hardcoded CT fiducial coordinates in LPS (mm)
CT_NOSE_MARKER_NAME = "F"
CT_NOSE_POSITION_LPS = np.array([
    15.04228687286377,
    86.46243651882577,
    114.52446880550107,
], dtype=float)

CT_PEDESTAL_POSITIONS_LPS = {
    "F_1": np.array([0.0031134106684476137, 47.44228744506836, 150.02127075195313], dtype=float),
    "F_2": np.array([23.697498321533203, 46.341487884521484, 147.47947692871094], dtype=float),
    "F_3": np.array([11.59162425994873, 39.69370450659213, 155.08550427615032], dtype=float),
    "F_4": np.array([12.013157142401923, 36.49899038310201, 178.17802063449454], dtype=float),
}

# ========== CT PEDESTAL CONFIGURATION (OPTIONAL) ==========
# The CT fiducial coordinates are embedded below. Set nose_landmark_name to the
# tracked nose landmark name (e.g., "F" if your 3D skeleton keeps the CT label).
# Set include_ct_pedestals=False to skip adding the four pedestal joints.
nose_landmark_name = "F"    # update to your tracked nose joint name, or set to None to skip
include_ct_pedestals = True  # set False to disable pedestal augmentation
# =========================================================


# ===================== HELPERS =====================

def load_3d_tracks_any(path_3dfile):
    """
    Returns:
        pts3d: (T,J,3) float array in the reference/world frame
        joint_names: list[str] length J
    Assumptions:
    - For .h5: dataset "tracks" with shape (T,K,J,3) -> take K=0
    - For .npz: either one big (T,J,3) or per-joint arrays
    - Optionally computes and appends pedestal from CT markers if configured
    """
    ext = os.path.splitext(path_3dfile)[1].lower()

    if ext in [".h5", ".hdf5"]:
        with h5py.File(path_3dfile, "r") as f:
            if "tracks" not in f:
                raise RuntimeError("3D H5: expected dataset 'tracks' not found.")
            raw = np.array(f["tracks"])  # expected (T,K,J,3)
        if raw.ndim != 4 or raw.shape[-1] != 3:
            raise RuntimeError(f"3D H5 'tracks' has shape {raw.shape}, expected (T,K,J,3).")

        T,K,J,_ = raw.shape
        pts3d = raw[:,0,:,:]  # (T,J,3)
        joint_names = [f"J{i+1}" for i in range(J)]

    elif ext == ".npz":
        z = np.load(path_3dfile, allow_pickle=True)
        keys = list(z.keys())

        # Try big (T,J,3)
        cand = [(k, z[k].shape) for k in keys
                if hasattr(z[k], "ndim")
                and z[k].ndim == 3
                and z[k].shape[-1] == 3]
        if cand:
            k_big, shp = sorted(cand, key=lambda kv: kv[1][1], reverse=True)[0]
            XYZ = np.array(z[k_big])  # (T,J,3)
            T,J,_ = XYZ.shape
            if "nodes" in z and len(z["nodes"]) == J:
                joint_names = [str(s) for s in z["nodes"]]
            else:
                joint_names = [f"J{i+1}" for i in range(J)]
            pts3d = XYZ
        else:
            # Else stitch separate arrays
            joint_blocks = []
            joint_names  = []
            for k in keys:
                arr = z[k]
                if not hasattr(arr,"ndim"):
                    continue
                if arr.ndim == 2 and arr.shape[1] == 3:
                    joint_blocks.append(np.array(arr)) # (T,3)
                    joint_names.append(k)
                elif arr.ndim == 3 and arr.shape[-1] == 3:
                    # e.g. (T,JJ,3)
                    T,JJ,_ = arr.shape
                    for jsub in range(JJ):
                        joint_blocks.append(np.array(arr)[:,jsub,:])
                        joint_names.append(f"{k}{jsub+1}")
            if not joint_blocks:
                raise RuntimeError("3D NPZ: no suitable 3D arrays found.")
            pts3d = np.stack(joint_blocks, axis=1)  # (T,J,3)
    else:
        raise RuntimeError(f"Unsupported 3D file extension: {ext}")

    # ========== ADD PEDESTALS IF CONFIGURED ==========
    if include_ct_pedestals and nose_landmark_name:
        try:
            ct_pedestal_data = compute_ct_pedestal_offsets()
            pedestal_offsets = ct_pedestal_data["pedestals"]
            print(
                f"[pedestal] Loaded {len(pedestal_offsets)} CT pedestal offsets "
                f"relative to nose '{ct_pedestal_data['nose_marker']}'"
            )

            # Find nose landmark in loaded joints
            nose_idx = None
            nose_key = nose_landmark_name
            if nose_key in joint_names:
                nose_idx = joint_names.index(nose_key)
            else:
                matching_indices = [i for i, n in enumerate(joint_names) if n.lower() == nose_key.lower()]
                if matching_indices:
                    nose_idx = matching_indices[0]
                    nose_key = joint_names[nose_idx]
                    print(f"[pedestal] Matched nose landmark: {nose_key}")
                else:
                    print(f"[pedestal] WARNING: Nose landmark '{nose_landmark_name}' not found in joints.")
                    print(f"[pedestal] Available joints: {joint_names}")
                    print(f"[pedestal] Skipping pedestal computation.")
                    return pts3d, joint_names

            # Get nose trajectory
            nose_traj = pts3d[:, nose_idx, :]  # (T, 3)

            # Compute pedestal trajectories
            pedestal_trajs = compute_pedestal_trajectories(nose_traj, pedestal_offsets)

            # Append pedestals to pts3d and joint names
            added_blocks = []
            added_joint_names = []
            for marker_name, traj in pedestal_trajs.items():
                added_blocks.append(traj[:, np.newaxis, :])
                added_joint_names.append(f"Pedestal_{marker_name}")

            if added_blocks:
                pts3d = np.concatenate([pts3d] + added_blocks, axis=1)
                joint_names.extend(added_joint_names)
                print(
                    f"[pedestal] Added pedestal trajectories computed from '{nose_key}': "
                    f"{', '.join(added_joint_names)}"
                )
                print("[pedestal] Pedestals will appear in reprojection error calculations")
        except Exception as e:
            print(f"[pedestal] ERROR: Failed to compute pedestals: {e}")
            print(f"[pedestal] Continuing without pedestals...")
    # ================================================

    return pts3d, joint_names


def load_2d_from_h5_analysis(path_2dfile):
    """
    YOUR FORMAT (from cam-*.inference.analysis.h5 dump):

    tracks shape = (1, 2, 20, 13107)
      axis 0: track index (we'll take 0)
      axis 1: coord = [x,y]
      axis 2: joint index (0..19)
      axis 3: frame index (0..T-1)

    We want pts2d[frame, joint, xy] = shape (T, J, 2).

    Steps:
      raw = tracks[0]            -> (2, J, T)
      swap axes -> (T, J, 2)
    Also returns node_names list for debugging.
    """
    with h5py.File(path_2dfile, "r") as f:
        if "tracks" not in f:
            raise RuntimeError(f"{path_2dfile}: no 'tracks' dataset")
        raw = np.array(f["tracks"])        # (1,2,J,T)
        node_names_ds = np.array(f["node_names"])  # (J,)

    if raw.ndim != 4:
        raise RuntimeError(f"{path_2dfile}: 'tracks' shape {raw.shape}, expected (1,2,J,T)")

    # squeeze first dim: (2,J,T)
    raw2 = raw[0]  # (2, J, T)

    # Now we want (T,J,2):
    # current axes: (coord=0, joint=1, frame=2)
    # move them to (frame, joint, coord) = (2,1,0)
    pts2d = np.moveaxis(raw2, [0,1,2], [2,1,0])  # -> (T,J,2)

    # decode node names to strings
    node_names = []
    for n in node_names_ds:
        if isinstance(n, (bytes, bytearray)):
            node_names.append(n.decode("utf-8"))
        else:
            node_names.append(str(n))

    return pts2d, node_names


# ========== CT MARKER OFFSETS (HARDCODED) ==========
def compute_ct_pedestal_offsets(ct_dir=None, nose_filename=None, pedestal_filenames=None):
    """
    Provide hardcoded CT offsets for the pedestal fiducials.

    Args:
        ct_dir: Unused (kept for backward compatibility).
        nose_filename: Unused.
        pedestal_filenames: Unused.

    Returns:
        Dictionary with:
            - "coordinate_system": LPS coordinate system string
            - "nose_marker": Nose marker name
            - "nose_position": Nose position [x, y, z] in mm
            - "pedestals": {
                marker_name: {
                    "offset": [dx, dy, dz] in mm,
                    "position": [x, y, z] in mm,
                    "filename": synthetic identifier
                }
              }
    """
    pedestals = {}
    for marker_name, pedestal_pos in CT_PEDESTAL_POSITIONS_LPS.items():
        offset = pedestal_pos - CT_NOSE_POSITION_LPS
        pedestals[marker_name] = {
            "offset": offset.tolist(),
            "position": pedestal_pos.tolist(),
            "filename": f"hardcoded::{marker_name}",
        }

    return {
        "coordinate_system": "LPS",
        "nose_marker": CT_NOSE_MARKER_NAME,
        "nose_position": CT_NOSE_POSITION_LPS.tolist(),
        "pedestals": pedestals,
    }


def compute_pedestal_trajectories(nose_trajectory, pedestal_offsets):
    """
    Compute pedestal trajectories from nose landmark trajectory and CT offsets.

    For each frame: pedestal_position = nose_position + pedestal_offset

    Args:
        nose_trajectory: Array of shape (T, 3) where T is number of frames,
            each row is [x, y, z] position of nose landmark
        pedestal_offsets: Dict mapping marker name -> {"offset": [dx, dy, dz], ...}

    Returns:
        Dict mapping marker name to array of shape (T, 3) with pedestal positions
        for each frame
    """
    if nose_trajectory.ndim != 2 or nose_trajectory.shape[1] != 3:
        raise ValueError(f"Expected nose_trajectory shape (T, 3), got {nose_trajectory.shape}")

    trajectories = {}
    for marker_name, info in pedestal_offsets.items():
        offset = np.array(info["offset"])
        if offset.shape != (3,):
            raise ValueError(f"Expected offset shape (3,) for '{marker_name}', got {offset.shape}")
        trajectories[marker_name] = nose_trajectory + offset

    return trajectories
# ================================================


def load_calibration(calib_json_path):
    """
    calibration.json structure includes:
      "P": { "cam-name.mp4": [[3x4], ...], ... }
      Optionally: "pedestal_config": { "ct_offset": [...], "nose_landmark_name": "..." }

    We only need P[camera] to project:
        [u,v,w]^T = P @ [X,Y,Z,1]^T
        x_pred = u/w
        y_pred = v/w
    """
    with open(calib_json_path, "r") as f:
        calib = json.load(f)

    if "P" not in calib:
        raise RuntimeError("calibration.json missing top-level 'P' block")

    cam_models = {}
    for cam_name, P_list in calib["P"].items():
        P = np.array(P_list, dtype=float)  # (3,4)
        if P.shape != (3,4):
            raise RuntimeError(f"P for {cam_name} has shape {P.shape}, expected (3,4)")
        cam_models[cam_name] = {"P": P}
    return cam_models


def reprojection_error_allcams(pts3d, joint_names_3d, cam_models, cam2d_files):
    """
    Compute reprojection error in pixels for each camera/joint.

    Inputs:
        pts3d           (T,J,3)
        joint_names_3d  list[str] len J for the 3D data
        cam_models      {camera: {"P":(3,4)}}
        cam2d_files     {camera: path_to_2d_h5}

    Returns:
        rows: list of dict rows with:
            camera, joint, median_reproj_px, p95_reproj_px, n_obs
    """
    T, J, _ = pts3d.shape
    rows = []

    for cam_name, h5path in cam2d_files.items():
        if cam_name not in cam_models:
            print(f"[warn] camera {cam_name} not in calibration; skipping")
            continue

        print(f"[step] loading 2D for {cam_name} from {h5path}")
        pts2d, node_names_2d = load_2d_from_h5_analysis(h5path)  # (T, J2, 2)
        # pts2d[frame, joint, xy], xy=(x,y) in pixels

        # Sanity: same frame count?
        if pts2d.shape[0] != T:
            raise RuntimeError(
                f"Frame mismatch {cam_name}: 3D has {T}, 2D has {pts2d.shape[0]}"
            )

        # Handle joint count mismatch (J2 could be different from J)
        J2 = pts2d.shape[1]
        if J2 < J:
            print(f"[warn] {cam_name}: 2D joints={J2} < 3D joints={J}; trimming 3D.")
            J_use = J2
        else:
            J_use = J

        P = cam_models[cam_name]["P"]  # (3,4)

        all_err = [[] for _ in range(J_use)]  # collect per-joint pixel errors

        # Loop frames
        for t in range(T):
            xyz = pts3d[t, :J_use, :]          # (J_use,3)
            ones = np.ones((J_use,1), float)
            xyz1 = np.concatenate([xyz, ones], axis=1)  # (J_use,4)

            proj = (P @ xyz1.T).T              # (J_use,3)
            u = proj[:,0] / proj[:,2]
            v = proj[:,1] / proj[:,2]
            pred2d = np.stack([u,v], axis=1)   # (J_use,2)

            gt2d = pts2d[t, :J_use, :]         # (J_use,2)

            diff = pred2d - gt2d               # (J_use,2)
            err = np.sqrt(np.sum(diff**2, axis=1))  # pixel distance
            good = np.isfinite(err)

            for j_idx in np.where(good)[0]:
                # Skip pedestal if it doesn't have 2D tracking data
                # (pedestal is computed from nose, so it won't be in 2D tracking)
                if j_idx < len(joint_names_3d) and joint_names_3d[j_idx] == "Pedestal":
                    # For pedestal, we can still compute reprojection but note it's computed, not tracked
                    # We'll still record it but it represents the projected position only
                    pass  # Include it anyway - it shows where pedestal projects to
                all_err[j_idx].append(err[j_idx])

        # summarize per joint for this camera
        for j_idx in range(J_use):
            errs = np.array(all_err[j_idx], float)
            errs = errs[np.isfinite(errs)]
            if errs.size == 0:
                med = np.nan
                p95 = np.nan
                nobs = 0
            else:
                med = float(np.median(errs))
                p95 = float(np.percentile(errs, 95))
                nobs = int(errs.size)

            # use 3D joint name if available
            if j_idx < len(joint_names_3d):
                joint_label = joint_names_3d[j_idx]
            else:
                joint_label = f"J{j_idx+1}"

            rows.append(dict(
                camera=cam_name,
                joint=joint_label,
                median_reproj_px=med,
                p95_reproj_px=p95,
                n_obs=nobs,
            ))

    return rows


# ===================== MAIN =====================

print("[step] loading 3D points ...")
pts3d, joint_names_3d = load_3d_tracks_any(path_3d)
print(f"[info] 3D shape: {pts3d.shape}, joints={joint_names_3d}")

print("[step] loading calibration ...")
cam_models = load_calibration(path_calib)
print(f"[info] cameras in calib:", list(cam_models.keys()))

print("[step] computing reprojection errors ...")
rows = reprojection_error_allcams(pts3d, joint_names_3d, cam_models, cam2d_files)

df = pd.DataFrame(
    rows,
    columns=["camera","joint","median_reproj_px","p95_reproj_px","n_obs"]
)

out_csv = os.path.join(out_root, "reprojection_error_by_cam_and_joint.csv")
df.to_csv(out_csv, index=False)
print("[OK] wrote reprojection CSV:", out_csv)

# Per-joint (across cameras) summary for quick view:
df_joint_summary = (
    df.groupby("joint")
      .agg(
          median_px_overall=("median_reproj_px","median"),
          p95_px_overall   =("p95_reproj_px","median"),
          total_obs        =("n_obs","sum"),
      )
      .reset_index()
)

out_csv2 = os.path.join(out_root, "reprojection_error_by_joint_overall.csv")
df_joint_summary.to_csv(out_csv2, index=False)
print("[OK] wrote joint summary CSV:", out_csv2)
print("[DONE]")


[step] loading 3D points ...
[info] 3D shape: (13107, 20, 3), joints=['J1', 'J2', 'J3', 'J4', 'J5', 'J6', 'J7', 'J8', 'J9', 'J10', 'J11', 'J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J19', 'J20']
[step] loading calibration ...
[info] cameras in calib: ['cam-topleft.mp4', 'cam-topright.mp4', 'cam-bottomleft.mp4', 'cam-bottomright.mp4']
[step] computing reprojection errors ...
[step] loading 2D for cam-bottomleft.mp4 from ../data/phase2_sample\cam-bottomleft.inference.analysis.h5
[step] loading 2D for cam-bottomright.mp4 from ../data/phase2_sample\cam-bottomright.inference.analysis.h5
[step] loading 2D for cam-topleft.mp4 from ../data/phase2_sample\cam-topleft.inference.analysis.h5
[step] loading 2D for cam-topright.mp4 from ../data/phase2_sample\cam-topright.inference.analysis.h5
[OK] wrote reprojection CSV: ../results\reprojection_error_by_cam_and_joint.csv
[OK] wrote joint summary CSV: ../results\reprojection_error_by_joint_overall.csv
[DONE]
